In [1]:
!pip install tensorflow-model-optimization
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 5.0 MB/s eta 0:00:00


In [2]:
# Load Oxford Flowers 102 dataset
dataset, info = tfds.load("oxford_flowers102", as_supervised=True, with_info=True)
num_classes = info.features['label'].num_classes

# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0  # Resize and normalize
    return image, label

# Prepare datasets
train_data = dataset['train'].map(preprocess).batch(32).shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_data = dataset['validation'].map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/6149 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.ABVLTT_2.1.1/oxford_flowers102-test.tfrecord*…

Generating validation examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.ABVLTT_2.1.1/oxford_flowers102-validation.tfr…

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


In [3]:
# Define a CNN model using MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the baseline model
model.fit(train_data, validation_data=val_data, epochs=5)

9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/5
32/32 [==============================] - 26s 272ms/step - loss: 4.2633 - accuracy: 0.1137 - val_loss: 3.0273 - val_accuracy: 0.3755
Epoch 2/5
32/32 [==============================] - 5s 111ms/step - loss: 2.0984 - accuracy: 0.6676 - val_loss: 2.0064 - val_accuracy: 0.6392
Epoch 3/5
32/32 [==============================] - 5s 109ms/step - loss: 1.1489 - accuracy: 0.8853 - val_loss: 1.5562 - val_accuracy: 0.7039
Epoch 4/5
32/32 [==============================] - 6s 130ms/step - loss: 0.7080 - accuracy: 0.9520 - val_loss: 1.3179 - val_accuracy: 0.7461
Epoch 5/5
32/32 [==============================] - 5s 112ms/step - loss: 0.4725 - accuracy: 0.9824 - val_loss: 1.1784 - val_accuracy: 0.7529


In [4]:
# Apply "Variational-Inspired" Pruning using Polynomial Decay
def apply_variational_pruning(model):
    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

    # Variational-inspired pruning schedule
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.1,   # Start with 10% sparsity
            final_sparsity=0.6,     # End with 60% sparsity
            begin_step=0,
            end_step=2000           # Gradual pruning
        )
    }

    pruned_model = prune_low_magnitude(model, **pruning_params)
    return pruned_model

# Apply pruning and recompile
pruned_model = apply_variational_pruning(model)
pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fine-tune pruned model
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep()
]

pruned_model.fit(train_data, validation_data=val_data, epochs=5, callbacks=callbacks)

Epoch 1/5
32/32 [==============================] - 45s 205ms/step - loss: 0.4099 - accuracy: 0.9598 - val_loss: 1.0682 - val_accuracy: 0.7578
Epoch 2/5
32/32 [==============================] - 6s 133ms/step - loss: 0.2010 - accuracy: 0.9951 - val_loss: 0.9338 - val_accuracy: 0.7814
Epoch 3/5
32/32 [==============================] - 7s 161ms/step - loss: 0.1252 - accuracy: 0.9980 - val_loss: 0.8794 - val_accuracy: 0.7951
Epoch 4/5
32/32 [==============================] - 12s 319ms/step - loss: 0.1836 - accuracy: 0.9951 - val_loss: 0.9434 - val_accuracy: 0.7765
Epoch 5/5
32/32 [==============================] - 6s 132ms/step - loss: 0.1162 - accuracy: 0.9980 - val_loss: 0.8971 - val_accuracy: 0.7804


In [5]:
# Evaluate pruned model
_, pruned_acc = pruned_model.evaluate(val_data)
print(f"Pruned Model Accuracy: {pruned_acc:.4f}")

32/32 [==============================] - 2s 61ms/step - loss: 0.8971 - accuracy: 0.7804
Pruned Model Accuracy: 0.7804
